<a href="https://colab.research.google.com/github/ManishaLokasani/Manisha_INFO5731_Spring2023/blob/main/Copy_of_In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

import requests
from bs4 import BeautifulSoup

# URL of the IMDB page for Cocaine Bear
url = "https://www.imdb.com/title/tt14209916/reviews/?ref_=tt_ql_2"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, "html.parser")

# Find all the user reviews on the page
reviews = soup.find_all("div", class_="lister-item-content")

# Print the text of each review
imdb_reviews = []
for review in reviews:
    text = review.find("div", class_="text").text.strip()
    imdb_reviews.append(text)
    print(text)





Ummmmm, where do I begin. I didn't watch any trailers or read any reviews. I saw the poster and title and thought, "I'm definitely watching that." After seeing it, I can say with confidence that it's exactly what a movie named Cocaine Bear should be.Aside from being an extremely fun and outrageous thriller, this movie is hysterical. I was full-volume laughing frequently. So many parts are the perfect combination of memorable and hilarious. It knows exactly what it's doing.The other thing that made me happy is seeing a studio have the courage to allow the filmmakers to go all out. I compare it to M3gan which was going for a similar tone. But M3gan was toned down to PG-13 so teens could watch. Cocaine Bear is rated R as it absolutely should be. It's not gratuitous but it never feels like it's holding back.I had a blast with this movie. If the title or premise intrigue you, you will most likely be pleased. (1 viewing, opening Thursday 2/23/2023)
Cocaine Bear promised so much but just didn

In [ ]:
df = pd.DataFrame({'user_reviews': imdb_reviews})

#save the dataframe on a csv file
df.to_csv('user_reviews.csv')
df.head()


,user_reviews
0,"Ummmmm, where do I begin. I didn't watch any t..."
1,Cocaine Bear promised so much but just didn't ...
2,I first groaned and rolled my eyes when I hear...
3,If you go down to the woods today you're sure ...
4,This film is exactly what you think it is. It'...


In [ ]:
import nltk
nltk.download('stopwords')
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version
install_java()
!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
!pip install --upgrade gensim
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import lxml.html as LH
import urllib.request as urllib2
from bs4 import BeautifulSoup
import spacy.cli
spacy.cli.download("en")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models  
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
data_word = df.user_reviews.tolist()
data_word


['Ummmmm, I begin. I watch trailers read reviews. I saw poster title thought, "I\'m definitely watching that." After seeing it, I say confidence exactly movie named Cocaine Bear be.Aside extremely fun outrageous thriller, movie hysterical. I full-volume laughing frequently. So many parts perfect combination memorable hilarious. It knows exactly doing.The thing made happy seeing studio courage allow filmmakers go out. I compare M3gan going similar tone. But M3gan toned PG-13 teens could watch. Cocaine Bear rated R absolutely be. It\'s gratuitous never feels like holding back.I blast movie. If title premise intrigue you, likely pleased. (1 viewing, opening Thursday 2/23/2023)',
 'Cocaine Bear promised much hit spot me.There many characters opinion becomes bit chore breaks things little much. If focus couple characters protagonists would better.The humour always work me. Sometimes good fell flat little often.The central premise great go far enough opinion. I feel like lot fun concept. Alt

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_word, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_word], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])





[['begin', 'watch', 'trailer', 'read', 'review', 'see', 'poster', 'title', 'think', 'definitely', 'watch', 'see', 'say', 'confidence', 'exactly', 'movie', 'name', 'cocaine', 'extremely', 'fun', 'outrageous', 'thriller', 'movie', 'hysterical', 'full', 'volume', 'laugh', 'frequently', 'many', 'part', 'perfect', 'combination', 'memorable', 'hilarious', 'know', 'exactly', 'thing', 'make', 'happy', 'see', 'studio', 'courage', 'allow', 'filmmaker', 'go', 'compare', 'go', 'similar', 'tone', 'tone', 'teen', 'watch', 'cocaine', 'rate', 'absolutely', 'gratuitous', 'never', 'feel', 'hold', 'blast', 'movie', 'title', 'premise', 'intrigue', 'likely', 'pleased', 'view', 'opening', 'thursday']]


In [ ]:

id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[:1])
[[(id2word[i], f) for i, f in c] for c in corpus[:1]]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 1), (55, 1), (56, 1), (57, 3)]]


[[('absolutely', 1),
  ('allow', 1),
  ('begin', 1),
  ('blast', 1),
  ('cocaine', 2),
  ('combination', 1),
  ('compare', 1),
  ('confidence', 1),
  ('courage', 1),
  ('definitely', 1),
  ('exactly', 2),
  ('extremely', 1),
  ('feel', 1),
  ('filmmaker', 1),
  ('frequently', 1),
  ('full', 1),
  ('fun', 1),
  ('go', 2),
  ('gratuitous', 1),
  ('happy', 1),
  ('hilarious', 1),
  ('hold', 1),
  ('hysterical', 1),
  ('intrigue', 1),
  ('know', 1),
  ('laugh', 1),
  ('likely', 1),
  ('make', 1),
  ('many', 1),
  ('memorable', 1),
  ('movie', 3),
  ('name', 1),
  ('never', 1),
  ('opening', 1),
  ('outrageous', 1),
  ('part', 1),
  ('perfect', 1),
  ('pleased', 1),
  ('poster', 1),
  ('premise', 1),
  ('rate', 1),
  ('read', 1),
  ('review', 1),
  ('say', 1),
  ('see', 3),
  ('similar', 1),
  ('studio', 1),
  ('teen', 1),
  ('thing', 1),
  ('think', 1),
  ('thriller', 1),
  ('thursday', 1),
  ('title', 2),
  ('tone', 2),
  ('trailer', 1),
  ('view', 1),
  ('volume', 1),
  ('watch', 3)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"movie" + 0.026*"go" + 0.020*"make" + 0.013*"cocaine" + 0.013*"get" + '
  '0.013*"good" + 0.013*"bad" + 0.013*"come" + 0.013*"hell" + 0.013*"almost"'),
 (1,
  '0.054*"movie" + 0.027*"bear" + 0.027*"want" + 0.027*"story" + '
  '0.018*"cocaine" + 0.018*"get" + 0.018*"go" + 0.018*"make" + 0.018*"watch" + '
  '0.018*"trailer"'),
 (2,
  '0.022*"film" + 0.022*"see" + 0.022*"story" + 0.021*"kill" + 0.015*"lot" + '
  '0.015*"kid" + 0.015*"half" + 0.015*"arc" + 0.015*"need" + 0.015*"much"'),
 (3,
  '0.036*"film" + 0.021*"make" + 0.021*"bear" + 0.014*"cocaine" + 0.014*"fun" '
  '+ 0.014*"give" + 0.014*"funny" + 0.014*"hear" + 0.014*"find" + '
  '0.014*"actual"'),
 (4,
  '0.001*"cocaine" + 0.001*"bear" + 0.001*"expect" + 0.001*"much" + '
  '0.001*"film" + 0.001*"make" + 0.001*"think" + 0.001*"movie" + '
  '0.001*"actually" + 0.001*"become"'),
 (5,
  '0.041*"movie" + 0.022*"make" + 0.022*"film" + 0.018*"really" + '
  '0.018*"horror" + 0.015*"scene" + 0.015*"fun" + 0.015*"funny" + 0.0

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Coherence score in topic modeling to measure how interpretable the topics are to humans.
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.3662187996819584
Coherence score with 3 clusters: 0.4598846262388472
Coherence score with 4 clusters: 0.42982349913577667
Coherence score with 5 clusters: 0.498510064341541
Coherence score with 6 clusters: 0.5406218565251717
Coherence score with 7 clusters: 0.4283194416105065
Coherence score with 8 clusters: 0.4325279376775685
Coherence score with 9 clusters: 0.5037025603248794
Coherence score with 10 clusters: 0.4531610994043483


In [ ]:
# perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)

In [ ]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

In [ ]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]
score1 = []
score2 = []
for doc in corpus_lsi:
    score1.append(round(doc[0][1],2))
    score2.append(round(doc[1][1],2))

# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = df['user_reviews']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)
df_topic.head(1)

,Text,Topic 0 score,Topic 1 score,Topic
0,"Ummmmm, I begin. I watch trailers read reviews...",4.7,-1.13,0


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
#import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


In [ ]:
df.head()

,user_reviews,Text (Clean)
0,"Ummmmm, I begin. I watch trailers read reviews...","[ummmmm, begin, watch, trailer, read, review, ..."
1,Cocaine Bear promised much hit spot me.There m...,"[cocain, bear, promis, hit, spot, there, chara..."
2,I first groaned rolled eyes I heard made film ...,"[groan, roll, ey, heard, film, bear, cocain, n..."
3,"If go woods today sure big surprise, mother be...","[wood, todai, sure, big, surpris, mother, bear..."
4,This film exactly think is. It's bear cocaine ...,"[film, exactli, think, bear, cocain, go, crazi..."


In [ ]:
import re
# Remove punctuation
df['user_reviews_processed'] = \
df['user_reviews'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['user_reviews_processed'] = \
df['user_reviews_processed'].map(lambda x: x.lower())
# Print out the first rows 
df['user_reviews_processed'].head()

0    ummmmm i begin i watch trailers read reviews i...
1    cocaine bear promised much hit spot methere ma...
2    i first groaned rolled eyes i heard made film ...
3    if go woods today sure big surprise mother bea...
4    this film exactly think is it's bear cocaine g...
Name: user_reviews_processed, dtype: object

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.user_reviews_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['ummmmm', 'begin', 'watch', 'trailers', 'read', 'reviews', 'saw', 'poster', 'title', 'thought', 'definitely', 'watching', 'seeing', 'say', 'confidence', 'exactly', 'movie', 'named', 'cocaine', 'bear', 'beaside', 'extremely', 'fun', 'outrageous', 'thriller', 'movie', 'hysterical', 'full', 'volume', 'laughing']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Write your code here

import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])




[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"bear" + 0.024*"movie" + 0.017*"cocaine" + 0.008*"film" + '
  '0.007*"horror" + 0.006*"funny" + 0.005*"could" + 0.005*"think" + '
  '0.005*"like" + 0.005*"story"'),
 (1,
  '0.013*"bear" + 0.011*"movie" + 0.008*"time" + 0.008*"funny" + '
  '0.008*"cocaine" + 0.007*"film" + 0.006*"comedy" + 0.006*"horror" + '
  '0.004*"fun" + 0.004*"make"'),
 (2,
  '0.017*"movie" + 0.014*"bear" + 0.010*"cocaine" + 0.009*"film" + 0.007*"get" '
  '+ 0.007*"characters" + 0.007*"want" + 0.005*"time" + 0.005*"see" + '
  '0.005*"watch"'),
 (3,
  '0.016*"movie" + 0.011*"bear" + 0.008*"fun" + 0.008*"funny" + 0.007*"comedy" '
  '+ 0.007*"really" + 0.006*"always" + 0.005*"cocaine" + 0.005*"good" + '
  '0.005*"well"'),
 (4,
  '0.019*"bear" + 0.015*"cocaine" + 0.015*"film" + 0.008*"movie" + 0.007*"fun" '
  '+ 0.007*"actually" + 0.006*"good" + 0.005*"much" + 0.005*"think" + '
  '0.004*"funny"'),
 (5,
  '0.028*"movie" + 0.014*"bear" + 0.013*"cocaine" + 0.009*"horror" + '
  '0.009*"comedy" + 0.008*"fun" +

In [ ]:
#import pyLDAvis.gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(LDAvis_prepared)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here





## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here





## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



